In [ ]:
class GaussianNaiveBayes:
    """
    
    """

    def __init__(self, eps=1e-8):
        """
       
        """
    def _gaussian_log_pdf(self, x, mean, var):
        """
        
        """

    def fit(self, X, y):
        """

        """
    def predict_log_proba(self, X):
        """
        
        """

    def predict(self, X):
        """
        
        """